## Database & Jupyter Notebook Setup
---

import the dataset: 
`mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [1]:
# import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# create an instance of MongoClient
mongo = MongoClient(port=27017)

# confirm db was created in MongoClient
mongo.list_database_names()

['admin',
 'artificats_db',
 'classDB',
 'config',
 'fruits_db',
 'gardenDB',
 'local',
 'petsityly_db',
 'uk_food']

In [3]:
# create db variable
db = mongo['uk_food']

# list collections in db
db.list_collection_names()

['establishments']

In [4]:
# create collection variable
establishments = db['establishments']

# review a document in the collection
pprint(establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677e14bc0e404675433fc5b1'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Update the Database
---

### Add new restaurant entry

In [5]:
# add a document for a new restaurant
establishments.insert_one(
    {
        "BusinessName":"Penang Flavours",
        "BusinessType":"Restaurant/Cafe/Canteen",
        "BusinessTypeID":"",
        "AddressLine1":"Penang Flavours",
        "AddressLine2":"146A Plumstead Rd",
        "AddressLine3":"London",
        "AddressLine4":"",
        "PostCode":"SE18 7DY",
        "Phone":"",
        "LocalAuthorityCode":"511",
        "LocalAuthorityName":"Greenwich",
        "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
        "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
        "scores":{
            "Hygiene":"",
            "Structural":"",
            "ConfidenceInManagement":""
        },
        "SchemeType":"FHRS",
        "geocode":{
            "longitude":"0.08384000",
            "latitude":"51.49014200"
        },
        "RightToReply":"",
        "Distance":4623.9723280747176,
        "NewRatingPending":True
    }
)

InsertOneResult(ObjectId('677e14e99d69611559c843e4'), acknowledged=True)

In [6]:
# check that new entry is there
new_entry_query = {"BusinessName":"Penang Flavours"}
establishments.count_documents(new_entry_query)

1

### Update BusinessTypeID for new entry

In [7]:
# find BusinessTypeID for BusinessType = Restaurant/Cafe/Canteen
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}
pprint(establishments.find_one(query, fields))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('677e14bc0e404675433fc5b1')}


In [8]:
# update new document with BusinessTypeID
establishments.update_one(
    new_entry_query,
    {'$set':{'BusinessTypeID': 1}}
)

# check if updated
pprint(establishments.find_one(new_entry_query))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677e14e99d69611559c843e4'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


### Remove entries within the Dover Local Authority

In [9]:
# check how many documents are within the Dover Local Authority
dover_query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents(dover_query)

994

In [10]:
# remove those entries
establishments.delete_many(dover_query)

# check to see they're gone
establishments.count_documents(dover_query)

0

### Update values from strings to numbers

##### Check if all values are numeric

In [11]:
# nonnumeric pattern (includes scientific notation)
nonnumeric_pattern = {'$not': {'$regex': r'^-?\d+(\.\d+)?([eE][+-]?\d+)?$'}}

# count documents with invalid form
print(f'Latitude: {establishments.count_documents({'geocode.latitude': nonnumeric_pattern})}')
print(f'Longitude: {establishments.count_documents({'geocode.longitude': nonnumeric_pattern})}')
print(f'RatingValue: {establishments.count_documents({'RatingValue': nonnumeric_pattern})}')

Latitude: 0
Longitude: 0
RatingValue: 4092


In [ ]:
# check values in RatingValue
establishments.distinct('RatingValue')

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 'Awaiting Inspection',
 'AwaitingInspection',
 'AwaitingPublication',
 'Exempt',
 'Pass']

##### Update non numeric values into null

In [13]:
# set non numeric values in RatingValue to Null
non_ratings = ['Awaiting Inspection', 'AwaitingInspection', 'AwaitingPublication', 'Exempt', 'Pass']

establishments.update_many(
    {'RatingValue': {'$in': non_ratings}},
    [
        {'$set': {'RatingValue': None}}
    ]
)

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

##### Update latitude, longitude, and RatingValue from string to numeric

In [ ]:
# update values to numeric values
establishments.update_many(
    {},
    [
        {
            '$set': {
                'geocode.latitude': {'$toDouble': '$geocode.latitude'},
                'geocode.longitude': {'$toDouble': '$geocode.longitude'},
                'RatingValue': {'$toInt': '$RatingValue'}
            }
        }
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [15]:
# check one entry
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677e14bc0e404675433fc898'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,
